xbloch simulations with stohr&scherz enhancement factor

In [1]:
from LB51.xbloch import sase_sim
from LB51.xbloch import do_xbloch_sim
from LB51.xbloch import enhancement_xbloch2020

from typing import List
import numpy as np 
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

N_PULSES = 20    # 20 for manuscript
N_TIME_POINTS = int(5e4)    # 5e4 for manuscript plot

In [2]:
%pdb
times_5fs = np.linspace(-25, 50, N_TIME_POINTS)
E_in_list = [sase_sim.simulate_gaussian(5.0, times=times_5fs)[1] for _ in range(N_PULSES)]

Automatic pdb calling has been turned ON


In [3]:
summary_result = do_xbloch_sim.simulate_multipulse_series(times_5fs, E_in_list, enhanced=True)

In [4]:
# save result to pickle for future reference
import pickle

results_file = 'LB51/xbloch/results/enhanced_5fs.pickle'
with open(results_file, 'wb') as f:
    pickle.dump(summary_result, f)

In [5]:
# load result
with open(results_file, 'rb') as f:
    loaded_result = pickle.load(f)

In [6]:
%matplotlib auto

for i, fluence in enumerate(summary_result['fluences']):
    plt.plot((loaded_result['summed_transmitted_intensities'][i]-loaded_result['summed_incident_intensities'][i])/loaded_result['fluences'][i], label=str(fluence))
plt.legend(loc='best')

Using matplotlib backend: MacOSX


In [7]:
# Make plot to put in manuscript
from LB51.manuscript_plots import set_plot_params
set_plot_params.init_paper_small()

f, axs = plt.subplots(2, 1, sharex=True, figsize=(3.37, 3.5))
incident = loaded_result['summed_incident_intensities'][0]
incident = incident/np.amax(incident)
axs[0].plot(loaded_result['phot'], incident)
inds_to_plot = [0, -7, -5, -3]
for i in inds_to_plot:
    intensity_difference = (loaded_result['summed_transmitted_intensities'][i]-loaded_result['summed_incident_intensities'][i])/loaded_result['fluences'][i]
    if i == 0:
        label = '10 nJ/cm$^2$'
        norm = np.amax(np.abs(intensity_difference))
    else:
        label = str(int(1E3*loaded_result['fluences'][i]))+' mJ/cm$^2$'
    axs[1].plot(loaded_result['phot'], intensity_difference/norm, label=label)
axs[0].set_xlim((772, 782))
axs[0].set_ylabel('Intensity (a.u.)')
axs[1].set_ylabel('Intensity (a.u.)')
axs[1].set_xlabel('Photon Energy (eV)')
axs[1].legend(loc='best', title='Fluence')
axs[0].text(
    0.9,
    0.9,
    "a",
    fontsize=10,
    weight="bold",
    horizontalalignment="center",
    transform=axs[0].transAxes,
)
axs[1].text(
    0.9,
    0.1,
    "b",
    fontsize=10,
    weight="bold",
    horizontalalignment="center",
    transform=axs[1].transAxes,
)
plt.tight_layout()

plt.savefig("plots/2020_09_18_enhancement_sim_overview.eps", dpi=600)
plt.savefig("plots/2020_09_18_enhancement_sim_overview.png", dpi=600)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [8]:
# make example plot of enhancement factor calculations
from LB51.xbloch import stohr_enhancement
intensities = np.logspace(9, 17, int(1E3))
factors = stohr_enhancement.calculate_factors(intensities)
plt.figure()
plt.semilogx(intensities, factors)
plt.xlabel('Intensities (W/cm$^2$)')
plt.ylabel('Enhancement Factor')

Text(0, 0.5, 'Enhancement Factor')